In [ ]:
cd ~/git/scim

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

import numpy as np
import anndata
from itertools import cycle

import scanpy as sc
from pathlib import Path

from scim.model import VAE, Integrator
from scim.discriminator import SpectralNormCritic
from scim.trainer import Trainer
from scim.utils import switch_obsm, make_network, plot_training, adata_to_pd
from scim.simulate import simulate
from scim.evaluate import score_divergence, extract_matched_labels, get_accuracy, get_confusion_matrix
from scim.matching import get_cost_knn_graph, mcmf

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
OUTDIR = Path('./.cache/demo')
SEED = 12
TECHS = ['source', 'target']
LABEL = 'branch'

In [ ]:
# PROSSTT params
ngenes = 200
ncells = 20000
tree = '(B:50,C:50)A:50'

In [ ]:
# Simulate and cache data

cache = {key: OUTDIR.joinpath('0_data', f'{key}.h5ad') for key in TECHS}

train = dict()
test = dict()
full = dict()

try:
    for tech in TECHS:
        path = cache[tech]
        data = anndata.read(path)
        full[tech] = data
        print(f'Loaded {path}')
        
except OSError:
    print('Error loading, recomputing data')
    np.random.seed(SEED)
    for tech in TECHS:
        data = simulate(tree, ngenes, ncells)
        full[tech] = data
        
        path = cache[tech]
        data.write(cache[tech])
        print(f'Cached {path}')


for tech, data in full.items():
    train[tech] = data[data.obs['is_train']]
    test[tech] = data[~data.obs['is_train']]

# Convert train into tf.data.Datasets
for tech, val in train.items():
    train[tech] = tf.data.Dataset.from_tensor_slices((val.X, val.obs[LABEL].cat.codes, val.obs_names.astype(int)))
    train[tech] = train[tech].batch(256)

# Need to know what the labels are
label_cats = None
for data in full.values():    
    # assert label categories is same on all technologies
    if label_cats is None:
        label_cats = data.obs[LABEL].cat.categories
    else:
        assert label_cats.equals(data.obs[LABEL].cat.categories)

In [ ]:
# Visualize simulated data

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
for idx, key in enumerate(TECHS):
    for jdx, color in enumerate(['branch', 'pt']):
        ax = axes[idx, jdx]
        sc.pl.pca(full[key], color=color, ax=ax, show=False)
        ax.set_title(f'{key} {color}')
plt.show()

In [ ]:
# Set up the model
latent_dim = 8

discriminator_net = make_network(
    doutput=1,
    units=[8]*2,
    dinput=latent_dim + label_cats.size,
    batch_norm=False,
    name='discriminator')

discriminator = SpectralNormCritic(
    discriminator_net,
    input_cats=label_cats)

vae_lut = dict()
for tech in TECHS:
    encoder = make_network(
        doutput=2*latent_dim, 
        units=[32]*2,
        dinput=ngenes,
        batch_norm=True, dropout=0.2,
        name=f'{tech}-encoder')

    decoder = make_network(
        doutput=ngenes,
        units=[32]*2,
        dinput=latent_dim,
        batch_norm=True, dropout=0.2,
        name=f'{tech}-decoder')

    vae_lut[tech] = VAE(
        encoder_net=encoder,
        decoder_net=decoder,
        name=f'{tech}-vae')
    
genopts = {key: tf.keras.optimizers.Adam() for key in TECHS}
disopt = tf.keras.optimizers.Adam()

trainer = Trainer(
    vae_lut=vae_lut,
    discriminator=discriminator,
    source_key='source',
    disopt=disopt,
    genopt_lut=genopts,
    beta=0.5)

In [ ]:
# Initialize the latent space by training a VAE on source technology

ckpt_dir = OUTDIR.joinpath('1_init', 'model')


source = 'source'
target = 'target'

try:
    trainer.restore(ckpt_dir)

except AssertionError:
    print('Initializing latent space by training VAE')

    np.random.seed(SEED)
    tf.compat.v1.random.set_random_seed(SEED)

    gs = 0
    for epoch in range(10):
        for (data, dlabel, _), (prior, plabel, _) in zip(train[source], cycle(train[target])):

            # Initializes latent space
            loss, (mse, kl), (codes, recon) = trainer.vae_step(source, data, beta=0.001)

            # Initialize the discriminator
            disc_loss, _ = trainer.discriminator_step(source, target, data, prior, dlabel, plabel)

            # Record
            if gs % 10 == 0:
                lut = {'loss': loss.numpy().mean(),
                       'mse': mse.numpy().mean(),
                       'kl': kl.numpy().mean()}
                trainer.record('vae', lut, step=gs)

            gs = gs + 1

    trainer.saver.save(ckpt_dir.joinpath('ckpt'))

    # Plot training curve
    fig, ax = plt.subplots()
    for key, pairs in trainer.history['vae'].items():
        step, vals = list(zip(*pairs))
        ax.plot(step, vals, label=key)
    ax.legend()
    ax.set_ylim(0, 2)
    plt.show()

else:
    print('Loaded checkpoint')

In [ ]:
# Visualize the initialized latent space

path = OUTDIR.joinpath('1_init', 'source_codes.h5ad')
path.parent.mkdir(exist_ok=True, parents=True)

try:
    codes = anndata.read(path)
    print(f'Loaded from cache {path}')
    
except OSError:
    print('Cache loading failed')
    
    trainer.forward('source', full['source'], LABEL)
    codes = switch_obsm(full['source'], 'code')
    sc.tl.pca(codes)
    sc.tl.tsne(codes, n_jobs=5)
    codes.write(path)
    print(f'Caching to {path}')
    

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
for idx, key in enumerate(['branch', 'pt']):
    sc.pl.pca(codes, color=key, ax=axes[0, idx], show=False)
    sc.pl.tsne(codes, color=key, ax=axes[1, idx], show=False)
plt.show()

In [ ]:
# Visualize discriminator performance on non-aligned latent space
path = OUTDIR.joinpath('1_init', 'evals.h5ad')
path.parent.mkdir(exist_ok=True, parents=True)

try:
    evald = anndata.read(path)
    print(f'Loaded from cache {path}')
    
except OSError:
    print('Cache loading failed')
    evald = trainer.evaluate(test, LABEL)
    sc.tl.tsne(evald, n_jobs=5)
    evald.write(path)
    print(f'Caching to {path}')
    
sc.pl.tsne(evald, color='probs-discriminator', color_map='PiYG', vmin=0, vmax=1)
sc.pl.tsne(evald, color='tech')

In [ ]:
# Integrate target to source codes
# This might take 5-10 mins

%%time

ckpt_dir = OUTDIR.joinpath('2_integrate', 'model')

source = 'target'
target = 'source'
try:
    trainer.restore(ckpt_dir)

except AssertionError:
    print('Training integration')
    np.random.seed(SEED)
    tf.compat.v1.random.set_random_seed(SEED)
    
    gs = 0
    for epoch in range(25):
        for (data, dlabel, didx), (prior, plabel, pidx) in zip(train[source], cycle(train[target])):

            # Train the discriminator
            for _ in range(trainer.niters_discriminator):
                disc_loss, _ = trainer.discriminator_step(source, target, data, prior, dlabel, plabel)

            # Fool adversary & reconstruct
            loss, (nll, adv), (codes, recon) = trainer.adversarial_step(source, data, dlabel)

            # Evaluate training batch, write to trainer.history
            if gs % 5 == 0:
                
                batch = full[source][didx.numpy()]
                trainer.forward(source, batch, LABEL)

                pbatch = full[target][pidx.numpy()]
                trainer.forward(target, pbatch, LABEL)

                lut = {f'mse-{source}': batch.obs['loss-mse'].mean(),
                       f'probs-{source}': batch.obs['probs-discriminator'].mean(),
                       f'probs-{target}': pbatch.obs['probs-discriminator'].mean(),
                       f'discriminator-{source}': batch.obs['loss-discriminator'].mean(),
                       f'discriminator-{target}': pbatch.obs['loss-discriminator'].mean(),
                      }
                trainer.record(f'train', lut, gs)

            # Evaluate testset
            if gs % 50 == 0:
                evald = trainer.evaluate(test, LABEL)
                probs = evald.obs.groupby('tech')['probs-discriminator'].mean()
                dloss = evald.obs.groupby('tech')['loss-discriminator'].mean()
                mse = evald.obs.groupby('tech')['loss-mse'].mean()

                lut = {'divergence': evald.uns['divergence']}
                lut.update({f'probs-{k}': v for k, v in probs.to_dict().items()})
                lut.update({f'mse-{k}': v for k, v in mse.to_dict().items()})
                lut.update({f'discriminator-{k}': v for k, v in dloss.to_dict().items()})

                trainer.record('test', lut, gs)

            gs = gs + 1

        trainer.saver.save(ckpt_dir.joinpath('ckpt'))
        fig, axes = plt.subplots(3, 1, figsize=(5, 9))
        plot_training(trainer, axes)
        
else:
    print('Loaded checkpoint')

In [ ]:
# Integrated latent space

cache = OUTDIR.joinpath('2_integrate', 'evals.h5ad')
try:
    evald = anndata.read(cache)
    print(f'Read from {cache}')
    
except OSError:
    print('Cache loading failed')
    evald = trainer.evaluate(test, LABEL)
    sc.tl.pca(evald)
    sc.tl.tsne(evald, n_jobs=5)
    
    evald.write(cache)

fig, axes = plt.subplots(2, 2, figsize=(12, 8), gridspec_kw={'wspace':0.5})
for color, ax in zip(['tech', 'branch', 'probs-discriminator', 'pt'], axes.ravel()):
    kwargs = {}
    if color == 'probs-discriminator':
        kwargs = {'color_map': 'PiYG', 'vmin':0, 'vmax':1}
    sc.pl.tsne(evald, color=color, ax=ax, show=False, **kwargs)

In [ ]:
# Visualize integrated latent space

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
for key, axl in zip(TECHS, axes):
    mask = evald.obs['tech'] == key
    for color, ax in zip(['branch', 'pt'], axl):
        sc.pl.tsne(evald[mask], color=color, ax=ax, show=False, **kwargs)

In [ ]:
# Cache model outputs
trainer.forward(source, full[source], LABEL)
trainer.forward(target, full[target], LABEL)

scode = switch_obsm(full[source], 'code')
tcode = switch_obsm(full[target], 'code')
inter = scode.concatenate(tcode,
                          batch_categories=[source, target],
                          batch_key='tech')


full[source].write(OUTDIR.joinpath('2_integrate', f'{source}.h5ad'))
full[target].write(OUTDIR.joinpath('2_integrate', f'{target}.h5ad'))
inter.write(OUTDIR.joinpath('2_integrate', f'codes.h5ad'))

In [ ]:
# Perform bipartite matching on the latent embeddings
source_pd = adata_to_pd(scode, add_cell_code_name='source')
target_pd = adata_to_pd(tcode, add_cell_code_name='target')

# Build an extended knn graph with k = 10% cells
G = get_cost_knn_graph(source_pd, target_pd, knn_k=20, null_cost_percentile=95, capacity_method='uniform')

# Run mcmf and extract matches
row_ind, col_ind = mcmf(G)
matches = extract_matched_labels(scode.obs, tcode.obs, row_ind, col_ind)


In [ ]:
# Cache matches
matches.to_csv(OUTDIR.joinpath('3_matching', f'k20_nullcost95_uniform.csv'))

In [ ]:
# Evaluate matches based on branch label
accuracy = get_accuracy(matches, colname_compare='branch')
print('accuracy (branch): ', accuracy)
get_confusion_matrix(matches, colname_compare='branch')